<a href="https://colab.research.google.com/github/Rutuja-Kolte/CodeBrewers/blob/master/MediaPlayerController.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import pandas as pd
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ln -s "/content/drive/My Drive/CodeBrewers" "/content/CodeBrewers"

In [4]:
model = keras.models.load_model('/content/CodeBrewers/GestureRecognition.h5')

In [5]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      //const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      //document.body.appendChild(div);
      //div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);



      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      //div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
gesture = ("down", "palm", "l", "fist", "fist_moved", "thumb", "index", "ok", "palm_moved", "c")
filename = take_photo()
img = cv2.imread('/content/photo.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (320, 120))
img = np.array(img)
img = np.reshape(img, (120,320,1))
averageValue1 = np.float32(img)
while True:
  try:
    filename = take_photo()
    img = cv2.imread('/content/photo.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (320, 120))
    img = np.array(img)
    img = np.reshape(img, (120,320,1))
    cv2.accumulateWeighted(img, averageValue1, 0.02)  
    resultingFrames1 = cv2.convertScaleAbs(averageValue1) 
    resultingFrames1 = np.reshape(resultingFrames1, (120,320,1))
    cv2_imshow(img)
    cv2_imshow(resultingFrames1)
    m = cv2.subtract(img,resultingFrames1)
    m = np.reshape(m, (120,320,1))
    cv2_imshow(m)
    prediction = model.predict(np.expand_dims(m, axis = 0))
    ans = np.argmax(prediction[0])
    print(prediction[0][ans])
    print(gesture[ans])
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

In [ ]:
averageValue1 = np.float32(img) 
  
# loop runs if capturing has been initialized.  
while(1): 
    # reads frames from a camera  
    _, img = cap.read() 
      
    # using the cv2.accumulateWeighted() function 
    # that updates the running average 
    cv2.accumulateWeighted(img, averageValue1, 0.02) 
      
    # converting the matrix elements to absolute values  
    # and converting the result to 8-bit.  
    resultingFrames1 = cv2.convertScaleAbs(averageValue1) 
  
    # Show two output windows 
    # the input / original frames window 
    cv2.imshow('InputWindow', img) 
  
    # the window showing output of alpha value 0.02 
    cv2.imshow('averageValue1', resultingFrames1)